<a href="https://colab.research.google.com/github/Andrey-Mel/parsing_hh/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_hh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1. Используя ноутбук занятия сделайте парсинг базы headhunter. Добавьте в конечную базу
готовность к переезду и командировкам (2 дополнительных столбца)

In [ ]:
import re #регулярные выражения
import pandas as pd #импортируем пандас для обработки и анализа данных
import datetime #импорт модуля datetime для обработки данных в формате даты
import numpy as np #импортируем библиотеку numpy
from google.colab import files #импортируем библиотеку files для работы с файлами

In [ ]:
#Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Считываем файл csv и сохраняем в датафрейм пандас
my_df = pd.read_csv('/content/drive/My Drive/hh.csv')

In [ ]:
my_df.head()

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,0,"Мужчина , 42 года , родился 6 октября 1976",27 000 руб.,Старший администратор,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяц\n\nСтарший администр...,Трц Ривьера,Старший администратор по сохраности активов,Среднее специальное образование 1994 Спту 26,28.04.2019 12:17,Имеется собственный автомобиль
1,1,"Мужчина , 41 год , родился 14 января 1978",60 000 руб.,Старший IT инженер,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяца\n\nСтарший IT инже...,"Эскорт-Центр, Москва, ЗАО",Старший инженер сервисного центра,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...,28.04.2019 11:44,Не указано
2,2,"Мужчина , 44 года , родился 21 февраля 1975",65 000 руб.,"Системный администратор, Системный инженер","Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцев\n\nСистемный адми...,"СПССПК ""ЭКОПТИЦА""",Системный администратор,Высшее образование 2003 Московский государств...,28.04.2019 10:36,Не указано
3,3,"Мужчина , 43 года , родился 9 февраля 1976",70 000 руб.,"Web-программист, Web-администратор, Web-мастер","Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы 5 лет 9 месяцев\n\nWeb-программист...,"ГБУ ""МДОО""",специалист отдела интернет-проектов,Высшее образование 2002 Российский государств...,28.04.2019 10:29,Не указано
4,4,"Мужчина , 39 лет , родился 14 июня 1979",45 000 руб.,Старший системный администратор \ начальник от...,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцев\n\nСтарший систем...,"ООО ""СервисСофт""",Старший системный администратор,Высшее образование 2007 Российский государств...,28.04.2019 10:15,Не указано


In [ ]:
my_df.shape

(66945, 13)

In [ ]:
n = 15
for i in range(len(my_df.values[n])):
  print(i,' ',my_df.values[n][i])

0   15
1   Мужчина ,  31 год , родился 1 сентября 1987
2   150 000 руб.
3   HighLoad, SDS Ceph,DevOps,SRE
4   Киров , готов к переезду , готов к командировкам
5    проектная работа, частичная занятость, полная занятость
6    удаленная работа, гибкий график, полный день, сменный график
7   Опыт работы 13 лет 6 месяцев

HighLoad, SDS Ceph,DevOps,SRE 150 000 руб. Информационные технологии, интернет, телеком Сетевые технологии Системный администратор Инженер Занятость: проектная работа, частичная занятость, полная занятость График работы: удаленная работа, гибкий график, полный день, сменный график Опыт работы 13 лет 6 месяцев Ноябрь 2017 — по настоящее время 1 год 6 месяцев Банк Хлынов Киров , www.bank-hlynov.ru/ Финансовый сектор ... Банк Ведущий инженер отдела системной интеграции 1. Ответственный за сектор сопровождения инфраструктуры. 2. Внедрение и обслуживание систем хранения данных Ceph (rbd,radosgw), суммарный объем 1ПБ 3. Внедрение и обслуживание систем резервного копирования 4. 

In [ ]:
#убираем столбец с продублироваными индексами
my_df = my_df.drop(my_df.columns[0], axis=1)

In [ ]:
#убрать кодировку \xa0 и \n
i = 0
for j in range(my_df.shape[1]):
  for i in range(my_df.shape[0]):
    if type(my_df.values[i][j]) != float:
      my_df.values[i][j] = my_df.values[i][j].replace('\xa0','')
      my_df.values[i][j] = my_df.values[i][j].replace('\n','')

In [ ]:
n = 3
for i in range(len(my_df.values[n])):
  print(i,' ',my_df.values[n][i])

0   Мужчина ,  43года , родился 9 февраля 1976
1   70000 руб.
2   Web-программист, Web-администратор, Web-мастер
3   Москва ,  м. Марьино , не готов к переезду , готов к редким командировкам
4    полная занятость
5    удаленная работа, гибкий график, полный день
6   Опыт работы 5 лет 9 месяцевWeb-программист, Web-администратор, Web-мастер 70000 руб. Информационные технологии, интернет, телеком Программирование, Разработка Web инженер Web мастер Занятость: полная занятость График работы: удаленная работа, гибкий график, полный день Опыт работы 5 лет 9 месяцев Июнь2017 — Август 2018 1 год 3 месяца ГБУ "МДОО" Москва , xn--80adfe5b7a9ayd.xn--80adxhks/ Государственные организации ... Государственные организации специалист отдела интернет-проектов Обязанности: доработка и поддержка в составе команды (сфера ответственности - frontend) основного проекта организации (yii2). Разработка и поддержка промо-страниц, промо-сайтов организации (WordPress, Bitrix). Сентябрь2016 — Май 2017 9 месяцев "Шал

In [ ]:
# Парсим базу
#Данные о поле и возрасте
def getParametersSexAge(arg):
  out = 0
  if 'Муж' in arg:
    sex = 'М'
  elif 'Жен' in arg:
    sex = 'Ж'
  else:
    sex = 'Не указан'
  #текущий год - год рождения
  year_tec = 2020
  if (len(arg) > 7):
    out = year_tec - int(re.findall(r'\d{4}', arg)[0])
  return sex, str(out)

  #Зарплата
def getParameterSalary(arg):
  global salaryStr, num 
  salaryStr = 0
  arg = str(arg)
  salaryStr = int(re.sub(r'[^0-9]+','',arg)) #удаляем НЕ числовые значения, + - любой длины,''- заменить на пустоту, arg - все это ищем в arg
  curr_ = re.sub('[0-9]','',arg).replace('.','')
  #Конвертируем в рубли
  if 'USD' in curr_: salaryStr = float(salaryStr)*75
  elif 'KZT' in curr_: salaryStr = float(salaryStr)*0.17
  elif 'грн' in curr_: salaryStr = float(salaryStr)*2.6
  num = salaryStr
  return salaryStr

#Данные о городе
def getParameterCity(arg):
  milionCities = "Новосибирск Екатеринбург Нижний Новгород Казань Челябинск Омск Самара Ростов-на-Дону Уфа Красноярск Пермь Воронеж Волгоград Сочи"
  #
  arg = arg.replace(' ','',3)
  arg = re.sub(r',м.',' м.', arg)

  #
  sarg = arg.split(',')
  item = sarg[0]
  #item = item.strip()
  pereezd = sarg[1]
  comandirovka = sarg[2]
  if item == 'Москва': return 'Москва', pereezd, comandirovka
  if item == 'Санкт-Петербург': return'Санкт-Петербург', pereezd, comandirovka
  if item in milionCities: return item , pereezd, comandirovka
  else: return  'малый или за пределами России город', pereezd, comandirovka
  
  
#Данные о желаемой занятости
def getParameterEmloyment(arg):
  result = ''
  if ('стажировка' in arg)|('volunteer' in arg)|('work placement' in arg): result += 'стажировка'
  if ('частичная занятость' in arg)|('part' in arg):  result += 'частичная занятость'
  if ('проектная работа' in arg)|('project' in arg): result += 'проектная работа'
  if ('полная занятость' in arg)|('full' in arg): result += 'полная занятость'
  if result == '': result = 'Не указано'
  return result

#Данные о желаемом графике работы
def getParameterSchedule(arg):
  result = ''
  if ('гибкий график' in arg)|('flexible' in arg): result += 'гибкий график'
  if ('полный день' in arg)|('full' in arg): result += 'полный день'
  if ('сменный график' in arg)|('shift' in arg): result += 'сменный график'
  if ('удаленная работа' in arg)|('remote' in arg): result += 'удаленная работа'
  if result == '': result = 'Не указано'
  return result

#Данные об образовании
def getParameterEducation(arg):
  result = ''
  arg = arg.lower()
  if ('высшее'in arg)|('higher education' in arg): result += 'высшее'
  if ('среднее специальное' in arg)|('special' in arg)|('колледж' in arg)|('college' in arg): result += 'среднее специальное'
  if ('неоконченное высшее' in arg)|('incomplete' in arg): result += 'неоконченное высшее'
  if ('среднее образование' in arg)|('secondary education' in arg): result += 'среднее образование'
  if result == '': result = 'Не указано'
  return result

#Данные об опыте работы
def getParameterExperience(arg):
  sarg = arg.split(' ', 5)
  year = 0
  month = 0
  if len(sarg)!=6: sarg.append(0);sarg.append("");sarg.append(0);sarg.append('')
  try:
    years = int(sarg[2])
  except ValueError:
    years = 0

  try:
    months = int(sarg[4])
  except ValueError:
    months = 0
  return 12*years+months

In [ ]:
my_df.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 42года , родился 6 октября 1976",27000 руб.,Старший администратор,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяцСтарший администратор...,Трц Ривьера,Старший администратор по сохраности активов,Среднее специальное образование 1994 Спту 26,28.04.2019 12:17,Имеется собственный автомобиль
1,"Мужчина , 41год , родился 14 января 1978",60000 руб.,Старший IT инженер,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяцаСтарший IT инженер ...,"Эскорт-Центр, Москва, ЗАО",Старший инженер сервисного центра,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...,28.04.2019 11:44,Не указано
2,"Мужчина , 44года , родился 21 февраля 1975",65000 руб.,"Системный администратор, Системный инженер","Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцевСистемный админист...,"СПССПК ""ЭКОПТИЦА""",Системный администратор,Высшее образование 2003 Московский государств...,28.04.2019 10:36,Не указано
3,"Мужчина , 43года , родился 9 февраля 1976",70000 руб.,"Web-программист, Web-администратор, Web-мастер","Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день","Опыт работы 5 лет 9 месяцевWeb-программист, We...","ГБУ ""МДОО""",специалист отдела интернет-проектов,Высшее образование 2002 Российский государств...,28.04.2019 10:29,Не указано
4,"Мужчина , 39лет , родился 14 июня 1979",45000 руб.,Старший системный администратор \ начальник от...,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцевСтарший системный ...,"ООО ""СервисСофт""",Старший системный администратор,Высшее образование 2007 Российский государств...,28.04.2019 10:15,Не указано


In [ ]:
df_my_p = my_df.drop(my_df.columns[[2,7,8,10,11]], axis=1) #удаляем не интересующие нас стоблцы

In [ ]:
df_my_p.head()

,"Пол, возраст",ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,"Мужчина , 42года , родился 6 октября 1976",27000 руб.,"Липецк , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 6 лет 1 месяцСтарший администратор...,Среднее специальное образование 1994 Спту 26
1,"Мужчина , 41год , родился 14 января 1978",60000 руб.,"Сочи , не готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день",Опыт работы 19 лет 2 месяцаСтарший IT инженер ...,Высшее образование 1999 СОЧИНСКИЙ ГОС. УНИВЕР...
2,"Мужчина , 44года , родился 21 февраля 1975",65000 руб.,"Липецк , готов к переезду , готов к командировкам",полная занятость,"удаленная работа, полный день, вахтовый метод",Опыт работы 19 лет 9 месяцевСистемный админист...,Высшее образование 2003 Московский государств...
3,"Мужчина , 43года , родился 9 февраля 1976",70000 руб.,"Москва , м. Марьино , не готов к переезду , г...",полная занятость,"удаленная работа, гибкий график, полный день","Опыт работы 5 лет 9 месяцевWeb-программист, We...",Высшее образование 2002 Российский государств...
4,"Мужчина , 39лет , родился 14 июня 1979",45000 руб.,"Тула , не готов к переезду , готов к редким ко...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 5 месяцевСтарший системный ...,Высшее образование 2007 Российский государств...


In [ ]:
#Создаем список в которых лежат объекты наших функций
funcList = [getParametersSexAge,getParameterSalary,getParameterCity,
            getParameterEmloyment, getParameterSchedule,getParameterExperience,
            getParameterEducation]

In [ ]:
for i in range(len(df_my_p.columns)):
  df_my_p[df_my_p.columns[i]] = [funcList[i] (cell) for cell in df_my_p[df_my_p.columns[i]]]
  

In [ ]:
df_my_p.head()

,"Пол, возраст",ЗП,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,"(М, 44)",27000.0,"(малый или за пределами России город, неготов ...",полная занятость,полный день,73,среднее специальное
1,"(М, 42)",60000.0,"(Сочи, неготов к переезду , готов к командиро...",частичная занятостьполная занятость,гибкий графикполный день,230,высшее
2,"(М, 45)",65000.0,"(малый или за пределами России город, готовк п...",полная занятость,полный деньудаленная работа,237,высшее
3,"(М, 44)",70000.0,"(малый или за пределами России город, не гото...",полная занятость,гибкий графикполный деньудаленная работа,69,высшее
4,"(М, 41)",45000.0,"(малый или за пределами России город, неготов ...",частичная занятостьполная занятость,полный деньсменный график,161,высшее


In [ ]:
#создаем еще колонки
df_my_p.insert(0,'Пол', np.array([df_my_p['Пол, возраст'][i][0] for i in range(len(df_my_p))]))
df_my_p.insert(1,'Возраст',np.array([df_my_p['Пол, возраст'][i][1] for i in range(len(df_my_p))]))
df_my_p.drop('Пол, возраст', axis=1, inplace=True)
df_my_p.insert(3,'Города', np.array([df_my_p['Город'][i][0] for i in range(len(df_my_p))]))
df_my_p.insert(4,'Переезд', np.array([df_my_p['Город'][i][1] for i in range(len(df_my_p))]))
df_my_p.insert(5,'Командировка', np.array([df_my_p['Город'][i][2] for i in range(len(df_my_p))]))
df_my_p.drop('Город', axis=1, inplace=True)


In [ ]:
df_my_p.head(20)

,Пол,Возраст,ЗП,Города,Переезд,Командировка,Занятость,График,Опыт (двойное нажатие для полной версии),Образование и ВУЗ
0,М,44,27000.0,малый или за пределами России город,неготов к переезду,не готов к командировкам,полная занятость,полный день,73,среднее специальное
1,М,42,60000.0,Сочи,неготов к переезду,готов к командировкам,частичная занятостьполная занятость,гибкий графикполный день,230,высшее
2,М,45,65000.0,малый или за пределами России город,готовк переезду,готов к командировкам,полная занятость,полный деньудаленная работа,237,высшее
3,М,44,70000.0,малый или за пределами России город,не готов к переезду,готов к редким командировкам,полная занятость,гибкий графикполный деньудаленная работа,69,высшее
4,М,41,45000.0,малый или за пределами России город,неготов к переезду,готов к редким командировкам,частичная занятостьполная занятость,полный деньсменный график,161,высшее
5,М,36,50000.0,малый или за пределами России город,готов к переезду,готов к командировкам,полная занятость,полный день,155,высшее
6,Ж,32,17000.0,малый или за пределами России город,неготова к переезду,не готова к командировкам,полная занятость,полный день,94,среднее специальное
7,М,41,80000.0,малый или за пределами России город,готов к переезду,готов к командировкам,полная занятость,полный день,200,высшее
8,Ж,42,120000.0,малый или за пределами России город,не готова к переезду,готова к командировкам,полная занятость,полный день,173,высшее
9,М,46,120000.0,малый или за пределами России город,готовк переезду (Москва),готов к командировкам,частичная занятостьпроектная работаполная заня...,гибкий графикполный деньсменный графикудаленна...,256,высшее


In [ ]:
#Сохраняю распаршеную базу
df_my_p.to_csv('/content/drive/My Drive/hh_parsed.csv', index= False)

Задание 2. Pro
Возьмите базу квартир (moscow.csv), проведите парсинг и очистку данных:
   ● Уберите строки “Для заметок”
   ● Уберите столбцы “ГРМ” и “Примечания”
   ● Для столбца “Балкон” добавьте категорию “Отсутствует” (вместо NaN)
   ● Значения столбца “Бонус агенту” переведите в тысячи рублей,а пустые значения
заполните нулями
   ● В столбец “Площадь” запишите общую площадь всех помещений
   ● Для столбца “Санузел” заполните отсутствующие значения строкой “Нет данных”
   ● Столбец “Цена” переведите в тысячи рублей



In [ ]:
rooms_M = pd.read_csv('/content/drive/My Drive/moscow.csv', sep=';' )

In [ ]:
rooms_M.head(5)

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
1,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
3,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."


In [ ]:
rooms_M.shape

(252536, 14)

In [ ]:
# 1) Уберите строки "Для заметок"
new_r = rooms_M[::2] # Отобразим с 10й по 20ю строки датафрейма с шагом 2

In [ ]:
new_r = new_r.reset_index(drop=True)

In [ ]:
new_r.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,NaN,NaN,25.07.2019,7.0,Привлекательные,"Предлагаю 1- комн квартиру, в шаговой доступно..."
4,1,Бабушкинская м.,15п,4/14 П,NaN,С,34/17/8,6250000.0,NaN,NaN,25.07.2019,11.0,Привлекательные,Продается хорошая 1 комнатная квартира. 1 взро...


In [ ]:
#Уберите столбцы “ГРМ” и “Примечания” 
new_r = new_r.drop(['ГРМ','Примечание'],axis=1)


In [ ]:
new_r.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,NaN,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,NaN,С,34/17/8,6250000.0,NaN,25.07.2019,11.0,Привлекательные


In [ ]:
#Для столбца “Балкон” добавьте категорию “Отсутствует” (вместо NaN)
new_r.Балкон = new_r.Балкон.fillna('Отсутствует') 
new_r.loc[new_r.Балкон == '-', 'Балкон'] = 'Отсутствует'

In [ ]:
new_r.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500000.0,NaN,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,NaN,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250000.0,NaN,25.07.2019,11.0,Привлекательные


In [ ]:
#Значения столбца “Бонус агенту” переведите в тысячи рублей,а пустые значения заполните нулями
#new_r['Бонус агенту'] = new_r['Бонус агенту'].fillna(1000) 
new_r.loc[new_r['Бонус агенту'].isna(), 'Бонус агенту'] = 0 # Зададим значение 0 для Nan

In [ ]:
b = new_r['Бонус агенту'] # Возьмем столбец "Бонус агенту"
c = new_r['Цена, руб.']    # Возьмем столбец 'Цена, руб.'

In [ ]:
for i in range(b.shape[0]): # Пройдем по всем значениям столбца "Бонус агенту"

  if isinstance(b[i], str) and b[i].find('руб.') != -1:  
    b[i] = b[i][:b[i].find('руб.')] 
    b[i] = b[i].replace(' ', '')        
    b[i] = float(b[i])                  
    b[i] = b[i] / 1000                  

  
  elif isinstance(b[i], str) and b[i].find('%') != -1: 
    b[i] = b[i][:b[i].find('%')] 
    b[i] = b[i].replace(' ', '')    
    b[i] = float(b[i])               
    b[i] = b[i] / 100                
    b[i] = b[i] * c[i]            
    b[i] = b[i] / 1000               
    
new_r['Бонус агенту'] = b 


In [ ]:
new_r.head() 

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250000.0,0,25.07.2019,11.0,Привлекательные


In [ ]:
#В столбец “Площадь” запишите общую площадь всех помещений
#sdf = new_r.copy()
new_r

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,64.1/23/20,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,38/15/7,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,34/17/8,6250000.0,0,25.07.2019,11.0,Привлекательные
...,...,...,...,...,...,...,...,...,...,...,...,...
126263,Ст,Некрасовка м.,4п,2/17 П,Отсутствует,NaN,19.8/?/?,2850000.0,0,12.07.2019,14.0,AVITO.ru
126264,Ст,Авиамоторная м.,6п,2/8 К,Отсутствует,С,20.4/18/?,3839990.0,0,23.07.2019,14.0,Яндекс
126265,Ст,Коломенская м.,15п,1/9 П,Отсутствует,NaN,15.4/13.2/?,2900000.0,0,12.07.2019,14.0,AVITO.ru
126266,Ст,Алтуфьево м.,5т,4/9 М,Отсутствует,NaN,26/10.9/7,5200000.0,0,22.07.2019,14.0,AVITO.ru


In [ ]:
s = []

for i in range(len(new_r.Площадь)):
  s = new_r.Площадь.values[i]
  s = s.replace('?','0')
  s = s.replace('/',',')
  s = re.sub(r'\d\d.\d\d.\d{4}','0,0,0',s)
  s = s.split(',')
  
  new_r.Площадь.values[i] = s 
  new_r.Площадь.values[i]=float(new_r.Площадь.values[i][0])+float(new_r.Площадь.values[i][1])+float(new_r.Площадь.values[i][2])
   

In [ ]:
new_r.Площадь

0         107.1
1          64.4
2            68
3            60
4            59
          ...  
126263     19.8
126264     38.4
126265     28.6
126266     43.9
126267     32.7
Name: Площадь, Length: 126268, dtype: object

In [ ]:
new_r.rename(columns={'Бонус агенту': 'Бонус агенту, тыс. руб.'}, inplace=True) # Даем новое название столбцу
new_r.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,107.1,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,64.4,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,68,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,60,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,59,6250000.0,0,25.07.2019,11.0,Привлекательные


In [ ]:
#Проверка столбца на не 0
new_r.loc[new_r['Бонус агенту, тыс. руб.'] != 0, 'Бонус агенту, тыс. руб.']

152          10
304          10
319          10
388          50
456        70.5
          ...  
124244       20
124510        5
124753    10.15
125287      150
125843       50
Name: Бонус агенту, тыс. руб., Length: 1252, dtype: object

In [ ]:
#Для столбца “Санузел” заполните отсутствующие значения строкой “Нет данных” 
new_r.loc[new_r.Санузел == '-', 'Санузел'] = 'Нет данных'
new_r['Санузел'] = new_r['Санузел'].fillna('Нет данных')

In [ ]:
new_r.head(10)

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,107.1,19500000.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,64.4,4800000.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,68,7990000.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,60,11200000.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,59,6250000.0,0,25.07.2019,11.0,Привлекательные
5,1,Шелепиха м.,15п,9/36 К,Отсутствует,Нет данных,77.9,12700000.0,0,25.07.2019,11.0,Привлекательные
6,1,Улица Дмитриевского м.,2п,7/16 П,Л,Р,71,6500000.0,0,25.07.2019,14.0,Привлекательные
7,1,Бульвар Рокоссовского м.,2п,2/5 К,Б,С,54,6000000.0,0,25.07.2019,15.0,Привлекательные
8,1,Марьина Роща м.,10п,7/14 П,2Б,Нет данных,68.3,8900000.0,0,25.07.2019,16.0,Привлекательные
9,1,Марьино м.,20п,10/24 М-К,Отсутствует,Нет данных,58.8,5999984.0,0,25.07.2019,16.0,Привлекательные


In [ ]:
#Столбец “Цена” переведите в тысячи рублей

new_r.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,107.1,19.50,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,64.4,4.80,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,68,7.99,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,60,11.20,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,59,6.25,0,25.07.2019,11.0,Привлекательные


In [ ]:
#Столбец “Цена” переведите в тысячи рублей

#df['Цена, руб.'] = df['Цена, руб.'] / 1000

m = []
for i in range(len(new_r['Цена, руб.'])):
  new_r['Цена, руб.'].values[i] = new_r['Цена, руб.'].values[i]/1000
  
new_r.rename(columns={'Цена, руб.': 'Цена, тыс. руб.'}, inplace=True)

In [ ]:
new_r.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.","Бонус агенту, тыс. руб.",Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,107.1,19500.0,0,25.07.2019,1.0,Привлекательные
1,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,64.4,4800.0,0,25.07.2019,2.0,Привлекательные
2,1,Планерная м.,5т,11/22 П,Б,С,68,7990.0,0,25.07.2019,4.0,Привлекательные
3,1,Шаболовская м.,12п,8/12 ?,Б,Р,60,11200.0,0,25.07.2019,7.0,Привлекательные
4,1,Бабушкинская м.,15п,4/14 П,Отсутствует,С,59,6250.0,0,25.07.2019,11.0,Привлекательные


In [ ]:
new_r.to_csv('/content/drive/My Drive/moskow_rooms_parsed.csv', index= False)